15.095. Homework 4

Kim-Anh-Nhi Nguyen

# Question 4

In [1]:
using JuMP, Gurobi, DataFrames

In [3]:
train_leaves = readtable("train_leaves.csv", header=false)
test_leaves = readtable("test_leaves.csv", header=false)
test_leaf_data = readtable("test_leaf_data_T.csv", header=false)
weights = readtable("weights.csv", header=false)
X_test = readtable("X_test_int.csv", header=true)
y_test = readtable("y_test.csv", header=true)
y_baseline = readtable("y_baseline.csv", header=true)
X_baseline = readtable("X_baseline.csv", header=true);

Stacktrace:
 [1] depwarn(::String, ::Symbol) at .\deprecated.jl:70
 [2] #readtable#232(::Bool, ::Char, ::Array{Char,1}, ::Char, ::Array{String,1}, ::Array{String,1}, ::Array{String,1}, ::Bool, ::Int64, ::Array{Symbol,1}, ::Array{Any,1}, ::Bool, ::Char, ::Bool, ::Int64, ::Array{Int64,1}, ::Bool, ::Symbol, ::Bool, ::Bool, ::DataFrames.#readtable, ::String) at C:\Users\utilisateur\.julia\v0.6\DataFrames\src\deprecated.jl:1045
 [3] (::DataFrames.#kw##readtable)(::Array{Any,1}, ::DataFrames.#readtable, ::String) at .\<missing>:0
 [4] include_string(::String, ::String) at .\loading.jl:522
 [5] softscope_include_string(::Module, ::String, ::String) at C:\Users\utilisateur\.julia\v0.6\SoftGlobalScope\src\SoftGlobalScope.jl:66
 [6] execute_request(::ZMQ.Socket, ::IJulia.Msg) at C:\Users\utilisateur\.julia\v0.6\IJulia\src\execute_request.jl:193
 [7] (::Compat.#inner#6{Array{Any,1},IJulia.#execute_request,Tuple{ZMQ.Socket,IJulia.Msg}})() at C:\Users\utilisateur\.julia\v0.6\Compat\src\Compat.jl:18

Stacktrace:
 [1] depwarn(::String, ::Symbol) at .\deprecated.jl:70
 [2] #readtable#232(::Bool, ::Char, ::Array{Char,1}, ::Char, ::Array{String,1}, ::Array{String,1}, ::Array{String,1}, ::Bool, ::Int64, ::Array{Symbol,1}, ::Array{Any,1}, ::Bool, ::Char, ::Bool, ::Int64, ::Array{Int64,1}, ::Bool, ::Symbol, ::Bool, ::Bool, ::DataFrames.#readtable, ::String) at C:\Users\utilisateur\.julia\v0.6\DataFrames\src\deprecated.jl:1045
 [3] (::DataFrames.#kw##readtable)(::Array{Any,1}, ::DataFrames.#readtable, ::String) at .\<missing>:0
 [4] include_string(::String, ::String) at .\loading.jl:522
 [5] softscope_include_string(::Module, ::String, ::String) at C:\Users\utilisateur\.julia\v0.6\SoftGlobalScope\src\SoftGlobalScope.jl:66
 [6] execute_request(::ZMQ.Socket, ::IJulia.Msg) at C:\Users\utilisateur\.julia\v0.6\IJulia\src\execute_request.jl:193
 [7] (::Compat.#inner#6{Array{Any,1},IJulia.#execute_request,Tuple{ZMQ.Socket,IJulia.Msg}})() at C:\Users\utilisateur\.julia\v0.6\Compat\src\Compat.jl:18

In [4]:
# target 
y_pred = test_leaves[:, 1];
y_baseline = y_baseline[:unit_sales];

## Function to get elements in the leaf of y_pred[i]

In [5]:
function get_data_points_in_leaf(i) # i is in 1:100
    init = test_leaf_data[i][:]
    points = zeros(0)
    for j in 1:length(init)
        if init[j] != -1
            append!(points, init[j])
        end
    end
    return(points)
end;

# Optimization Model

In [6]:
# number of items
n = 100

100

In [7]:
function optimize_profit(Q, data = X_test, target = y_pred, prescription = true)
    m = Model(solver = GurobiSolver(OutputFlag = 0))

    # decision variables
    if prescription == false
        @variable(m, s[1:n])
    end
    
    @variable(m, z[1:n])
    @constraint(m, sum(z) <= Q)
    
    for i = 1:n
        @constraint(m, z[i] <= 0.1 * Q * data[:ondisplay][i])
        
        
        if prescription == false
            @constraint(m, s[i] >= 0)
            @constraint(m, s[i] <= target[i])
            @constraint(m, s[i] <= z[i])
        end
    end

    @constraint(m, z[1:n] .>= 0)
    
    
    # objective function
    profit = 0
    
    if prescription == true
        for i = 1:n
            sub_profit = 0
            weight = weights[1][i]
            
            points = get_data_points_in_leaf(i) # points from the training set that belong to the same leaf as y_pred[i]
            p = length(points)
            x = @variable(m, [1:p])
            
            price = data[:price][i]
            promo = data[:onpromotion][i]
            cost = data[:cost][i]
            perish = data[:perishable][i]
            
            for j in 1:p
                @constraint(m, x[j] <= points[j])
                @constraint(m, x[j] <= z[i])
                
                sub_profit += (price *(1 - 0.2 * promo) - cost) * x[j] - (z[i] -  x[j]) * cost * perish
                
            end
            sub_profit = sub_profit * weight
            profit += sub_profit
        end

    else
        for i = 1:n
            price = data[:price][i]
            promo = data[:onpromotion][i]
            cost = data[:cost][i]
            perish = data[:perishable][i]
            profit += (price *(1 - 0.2 * promo) - cost) * s[i] - (z[i] - s[i]) * cost * perish
        end
    end
    
    @objective(m, Max, profit)

    status = solve(m)

    stocks = getvalue(z)
    r = getobjectivevalue(m)
    
    final_profit = 0
    oracle_profit = 0
    for i in 1:n
        final_profit += (data[:price][i] * (1 - 0.2 * data[:onpromotion][i]) - data[:cost][i])* min(stocks[i], y_test[:unit_sales][i]) - 
                        (stocks[i] - min(stocks[i], y_test[:unit_sales][i])) * data[:cost][i] * data[:perishable][i]
        oracle_profit += (data[:price][i] * (1 - 0.2 * data[:onpromotion][i]) - data[:cost][i])* y_test[:unit_sales][i]
    end
    
    return(stocks, final_profit)
end;

# Get the decisions: what stocks levels do we need for each item?

In [8]:
oracle_profit = 0
for i in 1:n
    oracle_profit += (X_test[:price][i]*(1 - 0.2 * X_test[:onpromotion][i]) - X_test[:cost][i])* y_test[:unit_sales][i]
end
oracle_profit

615.998

In [9]:
# implement the prescriptive method for different values of Q
Qs = [1, 10, 50, 100, 500, 1000, 10000, 100000]

for Q in Qs
    println("----- RESULTS for Q = ", Q, " -----")
    
    # baseline model
    stocks_baseline, profit_base = optimize_profit(Q, X_test, y_baseline, false)
    #println("----- The baseline suggest we stock the following volumns for each item: -----\n", stocks_baseline)
    println("* Baseline profit: ", profit_base)
    println("* Optimality gap: ", oracle_profit - profit_base, "\n")
    
    # prescriptions
    stocks, profit = optimize_profit(Q, X_test, y_pred, true);
    #println("----- We should stock the following volumns for each item: -----\n", stocks)
    println("* Our profit: ", profit)
    println("* Optimality gap: ", oracle_profit - profit)
    # percentage gain in profit
    gain = (profit-profit_base)/profit_base
    println("* Percentage gain in profit: ", gain*100, " % \n\n")
    
end

----- RESULTS for Q = 1 -----
Academic license - for non-commercial use only
* Baseline profit: 3.4350000000000005
* Optimality gap: 612.5630000000001

Academic license - for non-commercial use only
* Our profit: 3.0112
* Optimality gap: 612.9868
* Percentage gain in profit: -12.337700145560419 % 


----- RESULTS for Q = 10 -----
Academic license - for non-commercial use only
* Baseline profit: 34.35
* Optimality gap: 581.648

Academic license - for non-commercial use only
* Our profit: 30.112
* Optimality gap: 585.8860000000001
* Percentage gain in profit: -12.337700145560417 % 


----- RESULTS for Q = 50 -----
Academic license - for non-commercial use only
* Baseline profit: 129.84
* Optimality gap: 486.158

Academic license - for non-commercial use only
* Our profit: 163.682
* Optimality gap: 452.31600000000003
* Percentage gain in profit: 26.06438693776955 % 


----- RESULTS for Q = 100 -----
Academic license - for non-commercial use only
* Baseline profit: 226.56200000000004
* Opt